In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
pip install pyproj

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 6.3 MB 14.5 MB/s 


In [ ]:
attr = "학원"

year = ["2015", "2016", "2017"]

import pyproj
import folium
import numpy as np
import pandas as pd
import os

attr_list = ["일반게임제공업"] # 나름대로 적으시면 됩니다.
year_list = ["2015", "2016", "2017"]

def project_array(coord,p1_type,p2_type):
    p1=pyproj.Proj(init=p1_type)
    p2=pyproj.Proj(init=p2_type)
    fx,fy=pyproj.transform(p1,p2,coord[:,0],coord[:,1])
    return np.dstack([fx,fy])[0]

for attr in attr_list:
    os.mkdir('/content/drive/MyDrive/기쁨코인/dataset/X_attributes_modify/'+attr+'/')
    for year in year_list:
        # path_1은 변환할 좌표정보만(나중에 concat)
        # path_1 ='/content/drive/MyDrive/기쁨코인/dataset/X_attributes/유원시설업(기타)/유원시설업(기타)_2015.csv'
        path_1 ='/content/drive/MyDrive/기쁨코인/dataset/X_attributes/'+attr+'/'+attr+'_'+year+'.csv'
        data_location = pd.read_csv(path_1, encoding='utf-8',usecols=["좌표정보(X)","좌표정보(Y)"])

        # path_2는 전체 데이터
        # path_2 ='/content/drive/MyDrive/기쁨코인/dataset/X_attributes/유원시설업(기타)/유원시설업(기타)_2015.csv'
        path_2 ='/content/drive/MyDrive/기쁨코인/dataset/X_attributes/'+attr+'/'+attr+'_'+year+'.csv'
        data = pd.read_csv(path_2, encoding='utf-8')
        
        coord=np.array(data_location)
        p1_type="epsg:2097"
        p2_type="epsg:4326"

        result=project_array(coord, p1_type, p2_type)

        data_location["longitude"]=result[:,0]
        data_location["latitude"]=result[:,1]

        data_location=data_location.drop(["좌표정보(X)","좌표정보(Y)"],axis=1)
        data=data.drop(["좌표정보(X)","좌표정보(Y)"],axis=1)

        data=pd.concat([data,data_location],axis=1)

        data.to_csv('/content/drive/MyDrive/기쁨코인/dataset/X_attributes_modify/'+attr+'/'+attr+'_'+year+'.csv', encoding='utf-8')

In [121]:
# nan에 있는 좌표 옮기기

import numpy as np
import pandas as pd
import math

attr = '학원'

year = '2017'

path_nan = '/content/drive/MyDrive/기쁨코인/dataset/nan/nan_'+attr+'_'+year+'.xlsx'

path_to = '/content/drive/MyDrive/기쁨코인/dataset/X_attributes_modify/'+attr+'/'+attr+'_'+year+'.csv'

df_from = pd.read_excel(path_nan)

df_from['Unnamed: 0']

df_to = pd.read_csv(path_to)

# pd_to[math.isinf(pd_to['latitude'])]

idx_nan = df_to[df_to['latitude']==math.inf].index # 좌표계 변환 중 "Inf"가 된 행의 인덱스만 골라냄

df_from

# idx_nan
# 

,Unnamed: 0,소재지전체주소,Latitude,Longitude
0,26.0,서울특별시 중구 필동3가 26번지 혜화관,37.558481,127.000407


In [81]:
import os

os.mkdir('/content/drive/MyDrive/기쁨코인/dataset/X_attributes_final/'+attr)

In [110]:
list(idx_nan)

[26]

In [122]:
df_to.loc[list(idx_nan), ['longitude', 'latitude']] 

,longitude,latitude
26,inf,inf


궁금한 게, 

df_to.loc[list(idx_nan), ['longitude', 'latitude']] = df_from.loc[:, ['Longitude', 'Latitude']] 

이런식으로 할당하면 왜 결과값이 제대로 안되고 NaN이 되는 건가? (여기서 직접 시도해보진 마세요 데이터 꼬일듯)

In [129]:
df_from.loc[:, ['Longitude', 'Latitude']]

,Longitude,Latitude
0,126.830747,37.576489
1,126.830747,37.576489
2,127.015992,37.574888
3,127.001678,37.575008
4,126.974746,37.573479
5,126.999025,37.579443
6,126.990710,37.572609
7,126.969784,37.562782
8,127.018689,37.560560
9,126.986414,37.560924


In [114]:
df_from

,Unnamed: 0,소재지전체주소
0,26.0,서울특별시 중구 필동3가 26번지 혜화관


In [123]:
for i, idx in enumerate(list(idx_nan)):
    # df_to.loc[idx, ['longitude', 'latitude']]
    df_to.loc[idx, 'longitude'] = df_from.loc[i, 'Longitude']
    df_to.loc[idx, 'latitude'] = df_from.loc[i, 'Latitude']

In [124]:
df_to.loc[list(idx_nan), ['longitude', 'latitude']]

,longitude,latitude
26,127.000407,37.558481


In [125]:
df_to.to_csv('/content/drive/MyDrive/기쁨코인/dataset/X_attributes_final/'+attr+'/'+attr+'_'+year+'.csv', encoding='utf-8')

In [128]:
# nan에 따로 결측치로 모아 좌표 변환한 것들 다시 원래자리로 옮기기

import numpy as np
import pandas as pd
import math
import os

# attr_list = ['']  # <-- 이제 해야 합니다.
year_list = ['2015', '2016', '2017']

for attr in attr_list:
    os.mkdir('/content/drive/MyDrive/기쁨코인/dataset/X_attributes_final/'+attr)
    for year in year_list:
        # path_nan = '/content/drive/MyDrive/기쁨코인/dataset/nan/nan_'+attr+'_'+year+'.xlsx'
        path_nan = '/content/drive/MyDrive/기쁨코인/dataset/nan/nan_df_cafe_'+year+'.xlsx'
        path_to = '/content/drive/MyDrive/기쁨코인/dataset/X_attributes_modify/'+attr+'/'+attr+'_'+year+'.csv'

        df_from = pd.read_excel(path_nan)
        df_to = pd.read_csv(path_to)

        idx_nan = df_to[df_to['latitude']==math.inf].index # 좌표계 변환 중 "Inf"가 된 행의 인덱스만 골라냄

        for i, idx in enumerate(list(idx_nan)):
            # df_to.loc[idx, ['longitude', 'latitude']]
            df_to.loc[idx, 'longitude'] = df_from.loc[i, 'Longitude']
            df_to.loc[idx, 'latitude'] = df_from.loc[i, 'Latitude']

        df_to.to_csv('/content/drive/MyDrive/기쁨코인/dataset/X_attributes_final/'+attr+'/'+attr+'_'+year+'.csv', encoding='utf-8')
